In [31]:
# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
%matplotlib inline
from tensorflow.keras.utils import plot_model

# Splitting data
from sklearn.model_selection import train_test_split

# Metrics 
from sklearn.metrics import confusion_matrix, classification_report

# Deep Learning
import tensorflow as tf
print('TensoFlow Version: ', tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.applications.resnet import ResNet50

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

TensoFlow Version:  2.3.0


In [32]:
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten

import cv2 
import numpy as np 
from keras.datasets import cifar10 
from keras import backend as K 
from keras.utils import np_utils

import math 
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler

In [33]:
import pandas as pd
import numpy as np
import keras

In [36]:
train_path = 'cifar/train'
valid_path = 'cifar/test'

In [37]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [38]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 50000 images belonging to 10 classes.


In [39]:
X , y = training_set.next()

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)

In [41]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

In [42]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [43]:
input_layer = Input(shape=(224,224, 3))
x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(10, activation='softmax', name='auxilliary_output_1')(x1)

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(10, activation='softmax', name='auxilliary_output_2')(x2)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(10, activation='softmax', name='output')(x)

In [44]:
model = Model(input_layer, [x, x1, x2], name='inception_v1')

In [45]:
model.summary()

Model: "inception_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 112, 112, 64) 9472        input_4[0][0]                    
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling2D) (None, 56, 56, 64)   0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2a_3x3/1 (Conv2D)          (None, 56, 56, 64)   4160        max_pool_1_3x3/2[0][0]           
_______________________________________________________________________________________

In [48]:
epochs = 100
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])

In [49]:
history = model.fit(X_train, [y_train, y_train, y_train], validation_data=(X_test, [y_test, y_test, y_test]), epochs=epochs, batch_size=256, callbacks=[lr_sc])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 3.5122 - output_loss: 2.2134 - auxilliary_output_1_loss: 2.1012 - auxilliary_output_2_loss: 2.2280 - output_accuracy: 0.1600 - auxilliary_output_1_accuracy: 0.2800 - auxilliary_output_2_accuracy: 0.1600 - val_loss: 4.0927 - val_output_loss: 2.5354 - val_auxilliary_output_1_loss: 2.6174 - val_auxilliary_output_2_loss: 2.5735 - val_output_accuracy: 0.0000e+00 - val_auxilliary_output_1_accuracy: 0.0000e+00 - val_auxilliary_output_2_accuracy: 0.0000e+00

Epoch 00002: LearningRateScheduler reducing learning rate to 0.01.
Epoch 2/100
1/1 [==============================] - 0s 364ms/step - loss: 3.4849 - output_loss: 2.1560 - auxilliary_output_1_loss: 2.2908 - auxilliary_output_2_loss: 2.1388 - output_accuracy: 0.1600 - auxilliary_output_1_accuracy: 0.1600 - auxilliary_output_2_accuracy: 0.1600 - val_loss: 4.1399 - val_output_loss: 2.5869 - val_auxilliary_ou


Epoch 00015: LearningRateScheduler reducing learning rate to 0.0096.
Epoch 15/100
1/1 [==============================] - 0s 286ms/step - loss: 3.5974 - output_loss: 2.2803 - auxilliary_output_1_loss: 2.1502 - auxilliary_output_2_loss: 2.2402 - output_accuracy: 0.3200 - auxilliary_output_1_accuracy: 0.2400 - auxilliary_output_2_accuracy: 0.2000 - val_loss: 4.1552 - val_output_loss: 2.6597 - val_auxilliary_output_1_loss: 2.4528 - val_auxilliary_output_2_loss: 2.5323 - val_output_accuracy: 0.0000e+00 - val_auxilliary_output_1_accuracy: 0.0000e+00 - val_auxilliary_output_2_accuracy: 0.0000e+00

Epoch 00016: LearningRateScheduler reducing learning rate to 0.009216.
Epoch 16/100
1/1 [==============================] - 0s 335ms/step - loss: 3.5246 - output_loss: 2.2155 - auxilliary_output_1_loss: 2.1727 - auxilliary_output_2_loss: 2.1910 - output_accuracy: 0.2400 - auxilliary_output_1_accuracy: 0.1200 - auxilliary_output_2_accuracy: 0.0800 - val_loss: 4.0996 - val_output_loss: 2.6089 - val_au

1/1 [==============================] - 0s 346ms/step - loss: 3.5397 - output_loss: 2.2253 - auxilliary_output_1_loss: 2.1928 - auxilliary_output_2_loss: 2.1883 - output_accuracy: 0.2000 - auxilliary_output_1_accuracy: 0.2400 - auxilliary_output_2_accuracy: 0.3200 - val_loss: 4.1915 - val_output_loss: 2.6613 - val_auxilliary_output_1_loss: 2.5438 - val_auxilliary_output_2_loss: 2.5568 - val_output_accuracy: 0.0000e+00 - val_auxilliary_output_1_accuracy: 0.0000e+00 - val_auxilliary_output_2_accuracy: 0.0000e+00

Epoch 00029: LearningRateScheduler reducing learning rate to 0.008847359999999999.
Epoch 29/100
1/1 [==============================] - 0s 464ms/step - loss: 3.4750 - output_loss: 2.1991 - auxilliary_output_1_loss: 2.1314 - auxilliary_output_2_loss: 2.1216 - output_accuracy: 0.1600 - auxilliary_output_1_accuracy: 0.1600 - auxilliary_output_2_accuracy: 0.1200 - val_loss: 4.1990 - val_output_loss: 2.6659 - val_auxilliary_output_1_loss: 2.5390 - val_auxilliary_output_2_loss: 2.5710 -

1/1 [==============================] - 0s 334ms/step - loss: 3.4938 - output_loss: 2.1865 - auxilliary_output_1_loss: 2.1698 - auxilliary_output_2_loss: 2.1878 - output_accuracy: 0.2000 - auxilliary_output_1_accuracy: 0.2800 - auxilliary_output_2_accuracy: 0.1200 - val_loss: 4.1949 - val_output_loss: 2.6464 - val_auxilliary_output_1_loss: 2.5883 - val_auxilliary_output_2_loss: 2.5734 - val_output_accuracy: 0.0000e+00 - val_auxilliary_output_1_accuracy: 0.0000e+00 - val_auxilliary_output_2_accuracy: 0.0000e+00

Epoch 00042: LearningRateScheduler reducing learning rate to 0.008153726976.
Epoch 42/100
1/1 [==============================] - 0s 343ms/step - loss: 3.4275 - output_loss: 2.1505 - auxilliary_output_1_loss: 2.1392 - auxilliary_output_2_loss: 2.1175 - output_accuracy: 0.0400 - auxilliary_output_1_accuracy: 0.0800 - auxilliary_output_2_accuracy: 0.2400 - val_loss: 4.2103 - val_output_loss: 2.6581 - val_auxilliary_output_1_loss: 2.6046 - val_auxilliary_output_2_loss: 2.5695 - val_o


Epoch 00055: LearningRateScheduler reducing learning rate to 0.007827577896959998.
Epoch 55/100
1/1 [==============================] - 0s 352ms/step - loss: 3.3716 - output_loss: 2.1596 - auxilliary_output_1_loss: 2.0047 - auxilliary_output_2_loss: 2.0350 - output_accuracy: 0.2000 - auxilliary_output_1_accuracy: 0.2400 - auxilliary_output_2_accuracy: 0.2000 - val_loss: 4.2497 - val_output_loss: 2.6457 - val_auxilliary_output_1_loss: 2.7429 - val_auxilliary_output_2_loss: 2.6037 - val_output_accuracy: 0.0000e+00 - val_auxilliary_output_1_accuracy: 0.0000e+00 - val_auxilliary_output_2_accuracy: 0.0000e+00

Epoch 00056: LearningRateScheduler reducing learning rate to 0.007514474781081598.
Epoch 56/100
1/1 [==============================] - 0s 312ms/step - loss: 3.2120 - output_loss: 2.0308 - auxilliary_output_1_loss: 1.8853 - auxilliary_output_2_loss: 2.0522 - output_accuracy: 0.4000 - auxilliary_output_1_accuracy: 0.3200 - auxilliary_output_2_accuracy: 0.2400 - val_loss: 4.4124 - val_ou

1/1 [==============================] - 0s 373ms/step - loss: 2.9295 - output_loss: 1.8622 - auxilliary_output_1_loss: 1.7106 - auxilliary_output_2_loss: 1.8470 - output_accuracy: 0.3200 - auxilliary_output_1_accuracy: 0.3600 - auxilliary_output_2_accuracy: 0.3200 - val_loss: 5.7263 - val_output_loss: 3.7437 - val_auxilliary_output_1_loss: 3.3391 - val_auxilliary_output_2_loss: 3.2695 - val_output_accuracy: 0.0000e+00 - val_auxilliary_output_1_accuracy: 0.0000e+00 - val_auxilliary_output_2_accuracy: 0.0000e+00

Epoch 00069: LearningRateScheduler reducing learning rate to 0.007213895789838334.
Epoch 69/100
1/1 [==============================] - 0s 394ms/step - loss: 2.9122 - output_loss: 1.8128 - auxilliary_output_1_loss: 1.6859 - auxilliary_output_2_loss: 1.9787 - output_accuracy: 0.3200 - auxilliary_output_1_accuracy: 0.2800 - auxilliary_output_2_accuracy: 0.2000 - val_loss: 4.3688 - val_output_loss: 2.7357 - val_auxilliary_output_1_loss: 2.7917 - val_auxilliary_output_2_loss: 2.6521 -


Epoch 00082: LearningRateScheduler reducing learning rate to 0.006648326359915008.
Epoch 82/100
1/1 [==============================] - 0s 352ms/step - loss: 2.7751 - output_loss: 1.8605 - auxilliary_output_1_loss: 1.4571 - auxilliary_output_2_loss: 1.5917 - output_accuracy: 0.3200 - auxilliary_output_1_accuracy: 0.4400 - auxilliary_output_2_accuracy: 0.4800 - val_loss: 5.4291 - val_output_loss: 3.3871 - val_auxilliary_output_1_loss: 3.6114 - val_auxilliary_output_2_loss: 3.1952 - val_output_accuracy: 0.0000e+00 - val_auxilliary_output_1_accuracy: 0.0000e+00 - val_auxilliary_output_2_accuracy: 0.0000e+00

Epoch 00083: LearningRateScheduler reducing learning rate to 0.006648326359915008.
Epoch 83/100
1/1 [==============================] - 0s 322ms/step - loss: 2.8682 - output_loss: 1.9371 - auxilliary_output_1_loss: 1.4725 - auxilliary_output_2_loss: 1.6312 - output_accuracy: 0.2400 - auxilliary_output_1_accuracy: 0.4000 - auxilliary_output_2_accuracy: 0.4400 - val_loss: 5.2932 - val_ou

1/1 [==============================] - 0s 368ms/step - loss: 2.4039 - output_loss: 1.7062 - auxilliary_output_1_loss: 1.1170 - auxilliary_output_2_loss: 1.2086 - output_accuracy: 0.2400 - auxilliary_output_1_accuracy: 0.6400 - auxilliary_output_2_accuracy: 0.4400 - val_loss: 5.9448 - val_output_loss: 3.5325 - val_auxilliary_output_1_loss: 4.0317 - val_auxilliary_output_2_loss: 4.0094 - val_output_accuracy: 0.0000e+00 - val_auxilliary_output_1_accuracy: 0.0000e+00 - val_auxilliary_output_2_accuracy: 0.0000e+00

Epoch 00096: LearningRateScheduler reducing learning rate to 0.006127097573297671.
Epoch 96/100
1/1 [==============================] - 0s 427ms/step - loss: 2.4116 - output_loss: 1.7281 - auxilliary_output_1_loss: 0.9091 - auxilliary_output_2_loss: 1.3690 - output_accuracy: 0.2800 - auxilliary_output_1_accuracy: 0.6800 - auxilliary_output_2_accuracy: 0.3600 - val_loss: 6.2885 - val_output_loss: 3.7402 - val_auxilliary_output_1_loss: 4.2945 - val_auxilliary_output_2_loss: 4.1997 -